# Python and SQL databases


### Contents:
0. Install packages
1. Sqlite3 
2. MySQL (issue)
3. Mariadb (issue)
4. Duckdb

Please see also the notebook on InfluxDB in chapter 5 of this series.

## 0. Install packages

In [ ]:
%pip install mysql-connector-python

In [1]:
%pip --version

pip 21.2.4 from /Users/michielbontenbal/opt/anaconda3/lib/python3.9/site-packages/pip (python 3.9)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install duckdb

## 1. Sqlite3

Sqlite3 is a built in module in Python3, so you only have to import this module and it works out of the box. 

To use it, you'll need to make two objects:
1. a connection
2. a cursor, which is used to interact with the database.

We use the SQL language to interact with the database. See https://en.wikipedia.org/wiki/SQL for more info.
Or find an overview of SQL statements here: https://s3.amazonaws.com/assets.datacamp.com/email/other/SQL+for+Data+Science.pdf

Sqlite3 library automatically creates a database for us, if it does not exist already.
You can download a database viewer from https://sqlitebrowser.org/ to check your database.

In [2]:
import sqlite3
conn = sqlite3.connect('michiel.db') #database is created
cur = conn.cursor()
cur.execute('CREATE TABLE if not exists store(item TEXT, quantity INTEGER, price REAL)')
cur.execute("INSERT INTO store VALUES('fiets', 10, 1234.56)")
conn.commit()
conn.close()

In [3]:
conn = sqlite3.connect('michiel.db')
cur = conn.cursor()
cur.execute('SELECT * FROM store')
rows = cur.fetchall()
rows

[('fiets', 10, 1234.56), ('auto', 12, 10.5), ('fiets', 10, 1234.56)]

In [3]:
import sqlite3
conn = sqlite3.connect('michiel.db')
cur = conn.cursor()

def insert(item, quantity, price):
    cur.execute("INSERT INTO store VALUES(?,?,?)", (item, quantity, price))
    conn.commit()

insert('auto', 12, 10.5)
#conn.close()

In [32]:
#You can search the database by using LIKE
search = cur.execute("SELECT * FROM store WHERE item LIKE 'fiets'")
for x in search:
    print(x)

('fiets', 10, 1234.56)
('fiets', 10, 1234.56)


In [35]:
sql = "SELECT * FROM store WHERE item LIKE '"+ str('item')+"'"
print(sql)

SELECT * FROM store WHERE item LIKE 'item'


In [38]:
#You can search the database by using LIKE

def search(item):
    sql = "SELECT * FROM store WHERE item LIKE '"+ str(item)+"'"
    search = cur.execute(sql)
    #conn.commit()

print(sql)
search('fiets')
for x in search:
    print(x)

SELECT * FROM store WHERE item LIKE 'item'


TypeError: 'function' object is not iterable

## 2. MySQL connector Python

- pypi: https://pypi.org/project/mysql-connector-python/
- source: https://www.w3schools.com/python/python_mysql_getstarted.asp

### Install MySQL

Make sure you have installed MySQL on your device!
For Apple see this instruction using brew: https://flaviocopes.com/mysql-how-to-install/
For Windows, you can install it from here: https://dev.mysql.com/downloads/installer/        

In [1]:
#check if MySQL is installed, if this code runs without errors it is installed.
import mysql.connector

ModuleNotFoundError: No module named 'mysql'

In [3]:
import mysql.connector
import config #i've stored the password in the config file

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=config.mysql_pw
)

print(mydb)

In [4]:
#next create a cursor object to interact with the database and then crate a database
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE if not exists mydatabase")

In [5]:
#show databases
mycursor.execute("SHOW DATABASES")
for i in mycursor:
    print(i)

('information_schema',)
('mydatabase',)
('mysql',)
('performance_schema',)
('sys',)


In [6]:
#create a connection with the database selected
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=config.mysql_pw,
  database='mydatabase'   
)

In [40]:
#create a CURSOR object and with it create a table 'customers'
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE if not exists customers (name VARCHAR(255), address VARCHAR(255))")
print('done')

done


In [41]:
#show all tables
mycursor.execute("SHOW TABLES")
for x in mycursor:
    print(x)

('customers',)


In [13]:
# insert data into table
mycursor.execute("INSERT INTO customers VALUES('Henk', 'Wibautstraat')")
#mydb.commit()

In [16]:
#select records
mycursor.execute('SELECT * FROM customers')
rows = mycursor.fetchall()
rows

[('Henk', 'Wibautstraat', None)]

In [11]:
#delete record
sql = 'DELETE FROM customers WHERE name="Henk"'
mycursor.execute(sql)

In [26]:
#drop a table
mycursor.execute('DROP TABLE customers')

## 3. MariaDB

source: https://mariadb.com/resources/blog/how-to-connect-python-programs-to-mariadb/

In [2]:
!pip3 install mariadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 1.6 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [30 lines of output]
      /bin/sh: mariadb_config: command not found
      Traceback (most recent call last):
        File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 351, in <module>
          main()
        File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 333, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        File "/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/pip/_vendor/pep517/in_process/_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
        Fil

In [3]:
# Module Imports
import mariadb
import sys

# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="db_user",
        password="db_user_passwd",
        host="192.0.2.1",
        port=3306,
        database="employees"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

ModuleNotFoundError: No module named 'mariadb'

## 4. Duckdb
source: https://duckdb.org/docs/api/python/overview.html

In [3]:
#setting up a database
import duckdb
con = duckdb.connect(database='my-db.duckdb', read_only=False)

In [4]:
#PLEASE NOTE THAT YOU DON"T HAVE TO CREATE A CURSOR OBJECT!
# create a table
con.execute("CREATE TABLE items(item VARCHAR, value DECIMAL(10,2), count INTEGER)")
# insert two items into the table
con.execute("INSERT INTO items VALUES ('jeans', 20.0, 1), ('hammer', 42.2, 2)")

# retrieve the items again
con.execute("SELECT * FROM items")
print(con.fetchall())

[('jeans', Decimal('20.00'), 1), ('hammer', Decimal('42.20'), 2)]
